In [179]:
import numpy as np
import pandas as pd

In [180]:
from sklearn.model_selection import train_test_split
from sklearn.compose import ColumnTransformer
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import MinMaxScaler
from sklearn.pipeline import Pipeline,make_pipeline
from sklearn.feature_selection import SelectKBest,chi2
from sklearn.tree import DecisionTreeClassifier

In [181]:
df = pd.read_csv('https://raw.githubusercontent.com/campusx-official/100-days-of-machine-learning/refs/heads/main/day29-sklearn-pipelines/train.csv')

age and embarked have null values

- impute the missing
- one hot encode 
- scaling
- feature selection
- train DecisionTree 

In [182]:
df.drop(columns=['PassengerId', 'Name', 'Ticket', 'Cabin'], inplace=True)

In [183]:
X_train, X_test, y_train, y_test = train_test_split(df.drop(columns=['Survived']), df['Survived'], test_size=0.2, random_state=42)

In [184]:
X_train

,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked
331,1,male,45.5,0,0,28.5000,S
733,2,male,23.0,0,0,13.0000,S
382,3,male,32.0,0,0,7.9250,S
704,3,male,26.0,1,0,7.8542,S
813,3,female,6.0,4,2,31.2750,S
...,...,...,...,...,...,...,...
106,3,female,21.0,0,0,7.6500,S
270,1,male,NaN,0,0,31.0000,S
860,3,male,41.0,2,0,14.1083,S
435,1,female,14.0,1,2,120.0000,S


# Transformers

In [185]:
# Imputation transformer

trf1 = ColumnTransformer([
    ('impute_age', SimpleImputer(),[2]),
    ('impute_embarked', SimpleImputer(strategy='most_frequent'),[6])
], remainder='passthrough')

In [186]:
# One hot encoder

trf2 = ColumnTransformer([
    ('ohe_sex_embarked', OneHotEncoder(sparse_output=False, handle_unknown='ignore'),[1, 6])
], remainder='passthrough')

In [187]:
# Scaling

trf3 = ColumnTransformer([
    ('scaler', MinMaxScaler(), slice(0, 10)) # all columns
])

In [188]:
# Feature Selection

trf4 = SelectKBest(score_func=chi2, k=8)

In [189]:
# train model

trf5 = DecisionTreeClassifier()

### Pipeline Creation

In [190]:
pipe = Pipeline([
    ('trf1', trf1),
    ('trf2', trf2),
    ('trf3', trf3),
    ('trf4', trf4),
    ('trf5', trf5)
])

# Pipeline VS make_pipeline
- make_pipeline is easier
- Pipeline needs names of steps, it doesnt
- and with make_pipeline use make_column_transformer

In [191]:
pipe = make_pipeline(trf1, trf2, trf3, trf4 trf5)

SyntaxError: invalid syntax. Perhaps you forgot a comma? (1784514984.py, line 1)

In [ ]:
pipe.fit(X_train, y_train)

Pipeline(steps=[('columntransformer-1',
                 ColumnTransformer(remainder='passthrough',
                                   transformers=[('impute_age', SimpleImputer(),
                                                  [2]),
                                                 ('impute_embarked',
                                                  SimpleImputer(strategy='most_frequent'),
                                                  [6])])),
                ('columntransformer-2',
                 ColumnTransformer(remainder='passthrough',
                                   transformers=[('ohe_sex_embarked',
                                                  OneHotEncoder(handle_unknown='ignore',
                                                                sparse_output=False),
                                                  [1, 6])])),
                ('columntransformer-3',
                 ColumnTransformer(transformers=[('scaler', MinMaxScaler(),
                                                  slice(0, 10, None))])),
                ('decisiontreeclassifier', DecisionTreeClassifier())])

# Explore Pipeline

In [ ]:
pipe.named_steps

{'columntransformer-1': ColumnTransformer(remainder='passthrough',
                   transformers=[('impute_age', SimpleImputer(), [2]),
                                 ('impute_embarked',
                                  SimpleImputer(strategy='most_frequent'),
                                  [6])]),
 'columntransformer-2': ColumnTransformer(remainder='passthrough',
                   transformers=[('ohe_sex_embarked',
                                  OneHotEncoder(handle_unknown='ignore',
                                                sparse_output=False),
                                  [1, 6])]),
 'columntransformer-3': ColumnTransformer(transformers=[('scaler', MinMaxScaler(), slice(0, 10, None))]),
 'decisiontreeclassifier': DecisionTreeClassifier()}

In [ ]:
from sklearn import set_config
set_config(display='diagram')

In [192]:
pipe.fit(X_train, y_train)

Pipeline(steps=[('trf1',
                 ColumnTransformer(remainder='passthrough',
                                   transformers=[('impute_age', SimpleImputer(),
                                                  [2]),
                                                 ('impute_embarked',
                                                  SimpleImputer(strategy='most_frequent'),
                                                  [6])])),
                ('trf2',
                 ColumnTransformer(remainder='passthrough',
                                   transformers=[('ohe_sex_embarked',
                                                  OneHotEncoder(handle_unknown='ignore',
                                                                sparse_output=False),
                                                  [1, 6])])),
                ('trf3',
                 ColumnTransformer(transformers=[('scaler', MinMaxScaler(),
                                                  slice(0, 10, None))])),
                ('trf4',
                 SelectKBest(k=8,
                             score_func=<function chi2 at 0x0000028B5DE63740>)),
                ('trf5', DecisionTreeClassifier())])

In [193]:
y_pred = pipe.predict(X_test)

In [194]:
from sklearn.metrics import accuracy_score
accuracy_score(y_test, y_pred)

0.6256983240223464

# Cross Validation

In [195]:
from sklearn.model_selection import cross_val_score
cross_val_score(pipe, X_train, y_train, cv=5, scoring='accuracy').mean()

0.6391214419383433

# GridSearch

In [196]:
params = {
    'trf1__impute_age__strategy':['mean', 'median'],
    'trf5__max_depth':[1,2,3,4,5,None]
}

In [197]:
from sklearn.model_selection import GridSearchCV
grid = GridSearchCV(pipe, param_grid=params, cv=5, scoring='accuracy')
grid.fit(X_train, y_train)

GridSearchCV(cv=5,
             estimator=Pipeline(steps=[('trf1',
                                        ColumnTransformer(remainder='passthrough',
                                                          transformers=[('impute_age',
                                                                         SimpleImputer(),
                                                                         [2]),
                                                                        ('impute_embarked',
                                                                         SimpleImputer(strategy='most_frequent'),
                                                                         [6])])),
                                       ('trf2',
                                        ColumnTransformer(remainder='passthrough',
                                                          transformers=[('ohe_sex_embarked',
                                                                         OneHotEncoder(handle_unknown='ignore',
                                                                                       sparse_output=False),
                                                                         [1,
                                                                          6])])),
                                       ('trf3',
                                        ColumnTransformer(transformers=[('scaler',
                                                                         MinMaxScaler(),
                                                                         slice(0, 10, None))])),
                                       ('trf4',
                                        SelectKBest(k=8,
                                                    score_func=<function chi2 at 0x0000028B5DE63740>)),
                                       ('trf5', DecisionTreeClassifier())]),
             param_grid={'trf1__impute_age__strategy': ['mean', 'median'],
                         'trf5__max_depth': [1, 2, 3, 4, 5, None]},
             scoring='accuracy')

In [198]:
grid.best_score_

0.6391214419383433

In [199]:
grid.best_params_

{'trf1__impute_age__strategy': 'mean', 'trf5__max_depth': 2}

# Export Pipeline

In [200]:
import pickle
pickle.dump(pipe, open('pipe.pkl', 'wb'))